invertible에 대한 해석 통일  

1.2.1 설명 수정

부록 A

Path Guiding

https://arxiv.org/pdf/1605.08803.pdf  
https://arxiv.org/pdf/1808.03856.pdf

# Neural Importance Sampling

**Conributions**

• two piecewise-polynomial coupling transforms (piecewise-linear and piecewise-quadratic) that improve expressiveness,  
• one-blob-encoded network inputs—a generalization of onehot encoding—for improving learning speed and quality,  
• stochastic gradients that can be used for optimizing the $KL$ and $\chi^2$ divergences when only Monte Carlo estimates of the
unnormalized target distribution are available, and  
• an application of NICE with the aforementioned tools to the
problem of light-transport simulation

다음의 적분이 있을 때,

$$F = \int_{\mathcal D} f(x) dx \tag{1}$$

$q$를 상정하면 $F$를 integrand와 PDF의 expected ratio로 만들어 준다.

$$ \int_{\mathcal D} \frac{f(x)}{q(x)}q(x) \space dx = E\left[\frac{f(X)}{q(X)} \right] \tag{2}$$

위의 기댓값은 N개의 독립적으로 랜덤하게 선택된 점들 $\{X_1, X_2, \cdots, X_N \}$을 이용해서 다음의 MC estimator로 구할 수 있다.  
$X_i \in \mathcal D ,\quad X_i \sim q(x)$

$$F \approx \left<F \right>_N = \frac{1}{N} \sum_{i=1}^{N} \frac{f(X_i)}{q(X_i)} \tag{3}$$

여기서 Estimator의 분산은 $N$에 반비례하고, 또한 $q$의 형태에 따라 크게 달라진다. 만약 $q$가 normalized된 $f$를 잘 묘사하면 분산은 줄어들 것이고, 그렇지 않으면 분산이 커질 것이다.   
$f(x)$에 비례하는 PDF에서 샘플을 추출할 경우, 즉 $p(x) \equiv \frac{f(x)}{F}$, 우리는 $N \ge 1$일 때의 분산이 0인 Estimator $\left<F\right>_N =F$를 얻는다.

따라서 sampling density $q$가 **integrand $f$를 잘 표현**하는 것, **$X_i$를 빠르게 sampling**할 수 있는 것, 그리고 **invertible**한 것은, 좋은 sampling density를 위한 핵심 요소이다.

Generative Modeling은 일반적으로 $p(x, z; z) = p(x \space| \space z;\theta)\space p(z)$ 와 같은 probabilic model을 고려한다. 여기서 $z$는 직접적으로 관측되지 않지만 관측된 데이터 x의 일부 변동 요인을 제어하는 잠재 변수이다. $p(x \space|\space z; \theta)$는 $\theta$로 parametrized되는 likelihood function이고, $p(z)$는 prior이다. 그리고 우리의 주된 관심은 잠재 변수 $z$의 Posterior distribution $p(z\space|\space x; \theta)$이다. 관측된 데이터 x를 감안할 때, Posterior는 다음과 같은 Bayes 정리에 의해 정의된다.

$$p(z\space x; \theta) = \frac{p(x\space|\space z ; \theta)\space p(z)}{\int p(x\space|\space z ; \theta)\space p(z) \space dz} \tag{4}$$  

여기서 분모는 데이터 $p(x)$의 marginal likelihood이며 일반적으로 적분을 구하기 어렵다.

이 절에서는 본 논문의 토대가 되는 [Dinh 외 [2014, 2016]] 접근방식을 설명한다. 저자들은 역산이 가능한(invertible) 복합 함수 $\hat{h} = h_L \circ \cdots \circ h_2 \circ h_1$로서의 데이터와 잠재 공간 간의 mapping을 학습할 것을 제안했다. 여기서 각 $h_i$는 상대적으로 단순한 전단사 변환(bijective transformation)(warp)이다. (일대일 대응)  

$h$ 유형의 선택은 이전 두 작품과 본 논문에서 서로 다르지만(상세 사항은 Section 2에 있음), $h$는 (계산적으로) 다루기 쉬운 Jacobian들을 이용해 안정적으로 역산할 수 있어야 한다는 핵심 설계 원칙은 그대로이다.  

즉, 만약 $z \sim p_Z(z)$의 점에서 $x \in \mathcal X$로의 $\color{Magenta}{\text{미분 가능한 mapping}}$ $h :\mathcal Z \to \mathcal X$ 이 주어지면, 우리는 변환을 한 점 $x = h(z)$의 PDF $p_X(x)$를 다음의 변수 변환 공식에 의해 계산할 수 있다.

$$p_{\mathcal X}(x) = p_{\mathcal Z}(z) \space \left|\space det\left(\frac{\partial h(z)}{\partial z^T}\right)\right|^{-1} \tag{5}$$  

여기서 $\frac{\partial h(z)}{\partial z^T}$는 $z$에서 $h$의 Jacobian이다.

determinant를 계산하는 비용은 Jacobian의 차원성과 함께 기하급수로 증가한다. $\mathcal Z$와 $\mathcal X$가 고차원이라면 $p_{\mathcal X}(x)$는 계산적으로 다루기 어렵다. [Dinh[2014]]에서의 핵심적인 제안은 determinant가 diagonal항들의 곱으로 감소하는 Jacobian 행렬을 허용하는 mapping의 특정 Class(Coupling layer라고 함)에 초점을 맞추는 것이다.

## NON-LINEAR INDEPENDENT COMPONENTS ESTIMATION

### Coupling Layers

*Disjoint partition : 서로소의 합집합이 원래 집합이되는 부분집합

*Definition 1.1(Coupling layer)*  

Input vector를 $x \in R^D$로 놓고, A,B는 집합 [1,D]의 disjoint partitions로 놓자. 그리고 $m$을 $R^{|A|}$에 대한 function으로 놓자. 그러면 coupling layer의 output $y = (y^A, y^B) = h(x)$은 다음과 같이 정의된다.

$$y^A = x^A \tag{6}$$  
$$y^B = C(x^B; m(x^A)) \tag{7}$$

여기서 *coupling transform* $C \space : \space R^{|B|} \times \space m(R^{|A|}) \to R^{|B|}$ 은 **분리 가능**하고 **역산할 수 있는**  mapping이다. 

Coupling transform의 **invertibility**와, partition $A$가 변하지 않은 채로 남는다는 사실은 다음과 같이 Coupling layer의 역함수 $x = h^{-1}(y)$를 갖게 한다:  

$$x^A = y^A \tag{8}$$  
$$x^B = C^{-1}(y^B; m(y^A)) \tag{9}$$

Partition A가 임의로 변경될 수 있는 경우, 역함수(정확히는 eq(9)에서 m에 대한 input)는 찾기 어려울 것이다.  
MC integration에서 density estimation와 sample generation을 모두 요구하기 때문에, **invertibility**는 우리 환경에서 매우 중요하다.

Coupling layer $C$의 두 번째 중요한 속성은 **separability**이다. **separable** $C$는 Jacobian 행렬이 삼각행렬이고 Determinant는 대각항들의 곱으로 감소한다. [Dinh 외 [2014]] 또는 Appendix A 에서 자세히 다룬다. 따라서 Determinant의 계산은 $D$에 따라 선형적으로 감소되며, 따라서 고차원 문제에서도 다루기 쉽다(tractable).

### Affine Coupling Transforms

#### Additive Coupling Transform

[Dinh 외[2014]]은 단순히 $B$의 개별 차원에서만 신호를 해석하는 매우 단순한 Coupling transform을 설명한다.

$$C(x^B;t) = x^B+t \tag{10}$$

여기서 해석 벡터(translate vector) $t \in R^{|B|}$는 $m(x^A)$ function으로 생산된다. 

#### Multiply-add Coupling Transform

Additive Coupling layers에는 단위 Jacobian determinant가 있으므로(즉 volume을 보존), [Dinh 외[2016]]는 곱셈 계수 $e^S$를 추가할 것을 제안한다.

$$C(x^B;s,t) =x^B \odot e^s + t \tag{11}$$

여기서 $\odot$은 element-wise 곱이고 vectors $t$와 $s \in R^{|B|}$는 $m:\space (s,t) = m(x^A)$에 의해 생산된다.  
**Multiply-add coupling layer**의 Jacobian determinant는 단순히 $exp\sum s_i$이다.

위의 Coupling transforms는 비교적 간단하다. partitions 전체에 대해 nonlinear dependency를 학습할 수 있는 트릭은 parametric function $m$이다. 이 function은 Neural Net 등을 이용해 임의로 복잡하게 구성할 수 있다. Coupling layer의 역함수를 구하기위해 $m$의 역이 필요하지 않으며 $m$의 Jacobian은 Coupling layer의 determinant에 영향을 미치지 않기 때문이다. (부록 A 참조)

복잡한 $m$ function을 사용하면 두 partition 간에 복잡한 비선형 관계를 추출할 수 있다. Coupling transform $C$는 여전히 단순한 형태이고, invertible하며, 고차원 setting에서도 determinant에 다루기 쉬운 연산(tractable)을 허용한다.

### Compounding Multiple Coupling Layers

처음에 언급했듯이, 데이터 공간과 잠재 공간 사이의 완전한 변환은 다수의 coupling layers를 체인으로 연결함으로써 얻어진다.  
그리고 각 coupling layer에 대해 다른 신경망 $m$ instance를 학습한다. 모든 차원이 수정될 수 있도록 한 layer의 output은 아래의 **[그림 1]**처럼 두 partition의 역할을 교환한 채 다음 layer으로 공급된다. 이러한 방식으로 두 개의 coupling layers를 결합하면 모든 차원이 변경될 수 있다.

![figure 1](./IS1.png)

한 Coupling layer는 input $x$를 두 파티션 $A$와 $B$로 분할한다. 한 partition은 그대로 유지되지만 다른 partition의 차원(dimension)은 Neural Net $m$의 output에 의해 구동되는 Parametric Coupling transform $C$를 사용하여 뒤틀려 있다.  
이런 식으로 다수의 Coupling layers를 결합함으로써 설명력 있는 transform을 할 수 있게 된다.

각 차원이 다른 모든 차원에 영향을 미칠 수 있는지 확인하는 데 필요한 Coupling layers의 수는 총 차원 수에 따라 달라진다. 예를 들어, 2D setting(각 partition에 정확히 하나의 차원이 들어 있는 경우)에서는 2개의 layer만 필요하다. 3D 문제에는 3개의 layer가 필요하며, 다른 고차원 구성에는 최소한 4개의 layer가 있어야 한다.

그러나 실제로 고차원 문제(예: 얼굴 이미지 생성)는 각 affine transform으로는 상당히 제한적이기 때문에 훨씬 더 많은 Coupling layers를 필요로 한다. 다음 절에서는 Coupling layers의 수를 줄일 수 있는 더 많은 표현적 mapping을 제공하여 sample generation 및 Density estimation에 있어서의 계산 비용을 해결한다.  
그리고 이로써 4절에 제시된 MC estimator의 성능이 향상된다.

## PIECEWISE-POLYNOMIAL COUPLING LAYERS


이 절에서 우리는 앞서 보았던 제한된 affine warps 대신에 piecewise-polynomial invertible mapping을 Coupling layer로 제안한다. 구체적으로, 1차, 2차의 piecewise-polynomial(각 piecewise-linear and piecewise-quadratic) warps를 소개한다. 

$x,y \in (-\infty, +\infty)^D$ 및 Gaussian 잠재 변수를 가정하는 [Dinh 외 [2014, 2016]]와는 대조적으로,  
대부분의 실제 문제가 finite한 영역에 걸쳐 있기 때문에 Uniformly distributed 잠재 변수를 가진 unit hypercube(즉, $x,y \in [0, 1]^D$)에서 동작하는 것을 선택했다.

Unbounded domains 경우엔 $h_1$의 input과 $h_L$의 output을 예를 들면 sigmoid 및 logit function를 사용하여 뒤틀어서 처리할 수 있다.

Dinh 논문과 마찬가지로, 우리는 **separability**을 통해 계산적으로 다루기 쉬운(tractable) Jacobian들을 보장한다. 각 차원은 독립적으로 다음과 같이 변환된다:

$$C(x^B; m(x^A)) = \left( C_1(x_1^B; m(x^A)), \cdots, C_{|B|}\left(x_{|B|}^B; m(x^A)\right)\right)^T \tag{12}$$

단위 간격으로 작동시키는 것은 warp function $C_i$를 누적 분포 함수(CDF)로 해석할 수 있게 한다. 각 $C_i$를 생성하기 위해, 우리는 Neural Network를 관찰(instrument)하여 해당 unnormalized probability density $q_i$ 를 출력하고, integration에 의해 $C_i$를 구축한다.
이 설명은 아래 **그림 2**에 나타나 있다.

![그림 2](./IS2.png)

위의 그림은 $|B| = 4$일 때의 piecewise-quadratic transform을 하는 Coupling layer를 보여준다. Partition $A$(및 additional feature)의 신호는 one-blob encoding을 이용하여 encoded되고 fully connected layer로 U자형 Neural Network $m$에 공급된다. $m$의 output은 warping PDFs를 정의하는 산출 행렬 $V$와 $W$로 normalized된다. PDFs는 piecewise-quadratic coupling transforms를 얻기 위해 해석적으로 integrated된다. 그리고 이 piecewise-quadratic coupling transforms $x_B$의 각 차원을 warping시킨다. 

성능을 더욱 향상시키기 위해, Neural Network의 input을 one-blob encoding으로 encoding하는 것을 제안한다. 이에 대해선 Section 1.2.3에서 논한다.

### Piecewise-Linear Coupling Transform

$x = (x^A, x^B)$와 같이 $D$-차원 input 벡터를 $A$와 $B$라는 두 disjoint partition으로 분할한다는 점을 상기해보자. Partition $B$의 단위 차원들을 동일한 너비 $w = K^{-1}$의 $K$ bins로 나눈다. 모든 $|B|$ 변환을 한번에 정의하기 위해, 우리는 $\hat{Q}$로 표기된 $|B| \times K$ 행렬을 예측할 Neural Network $m(x^A)$를 관찰(instrument)한다.  
$\hat{Q}$의 각 $i$번째 행은 warp의 unnormalized PMF를 $x^B$의 i번째 차원으로 정의한다.(?) Softmax $\sigma$를 사용하여 행을 normalize하며, normalized된 행렬 $Q$를 $Q_i = \sigma(\hat{Q}_i)$와 같이 나타낸다.
(설명 수정 필요)

그러면 $i$번째 차원의 PDF는 $q_i(x_i^B) = \frac{Q_{ib}}{w}$와 같이 정의된다. 여기서 $b = \lfloor Kx_i^B \rfloor$는 스칼라 값 $x_i^B$를 담고 있는 bin이다. 이 PDF를 integrate하여 우리의 **invertible** piecewise-linear warp $C_i$를 다음과 같이 얻게 된다.

$$C_i(x_i^B;Q) = \int_{0}^{x_i^B} \space q_i(t)\space dt = \alpha Q_{ib} + \sum_{k=1}^{b-1} Q_{ik} \tag{13}$$

여기서 $\alpha = Kx_i^B - \lfloor Kx_i^B \rfloor$는 $b$안에서 $x_i^B$의 상대적 위치를 나타낸다

Coupling layer로부터 나온 density 결과의 변화를 평가하기 위해선, eq(5)에서처럼 density의 Jacobian이 필요하다. $C(x^B;Q)$가 정의상 **separable**하기 때문에 $C$의 Jacobian 행렬은 대각행렬이고 determinant는 diagonal 항들의 곱과 같다. 이것은 $Q$를 사용하여 계산될 수 있다.

$$det\left( \frac{\partial C(x^B; Q)}{\partial(x^B)^T}\right) = \prod_{i=1}^{|B|}q_i(x_i^B) = \prod_{i=1}^{|B|}\frac{Q_{ib}}{w} \tag{14}$$

여기서 $b$는 $i$번째 차원에서 값을 포함하는 bin을 나타낸다. bins의 수 $K$를 줄이기 위해선 Neural Network가 bin  width 또한 잘 예측하는 것을 의미한다. 이것은 불행히도 piecewise-linear 경우에선 gradient descent로 최적화해선 달성하기 어렵다(Appendix B).  
이 문제를 해결하고 정확성을 높이기 위선 piecewise-*quadratic* coupling transforms가 필요하다.

### Piecewise-Quadratic Coupling Transform

Piecewise-quadratic coupling transforms는 아래 그림의 왼쪽 하단처럼 $K + 1$의 꼭지점을 사용하여 모델링하는 Piecewise linear PDF를 허용한다.

![그림 3](./IS3.png)

예측된 Probability density functions(PDF, 왼쪽) 및 상응하는 $K = 5$개의 bin의 CDF(오른쪽)는 target distribution에 fitted된다. 상단 행은 piecewise linear의 CDF, 하단 행은 piecewise quadratic CDF를 나타낸다. Piecewise quadratic CDF의 1차 연속성($C^1$)과 adaptive bin sizing 덕분에 실전에서 더 잘 맞는 경향이 있다.  Appendix B에서 우리는 Piecewise quadratic CDF와 대조적으로 경사 기반 최적화 방법을 사용하는 부분적 선형 CDF의 경우 adaptive bin sizing을 달성하기 어렵다는 것을 보여준다.

![tmp](./tmp.png)

### One-Blob Encoding

중요한 고려사항은 Network에 대한 input의 encoding이다. 이 논문에선 One-blob encoding을 사용할 것을 제안한다. 즉, One-hot encoding[Harris & Harris 2013]의 일반화를 사용하여 커널을 단일 항목 대신 인접한 여러 항목을 활성화할 수 있다.  

스칼라 $s \in [0, 1]$ 및 단위 간격을 $k$개의 bin으로 quantization한다고 가정해보자(예 k = 32). 그러면 One-blob encoding은 커널($\sigma = 1/k$인 가우시안 사용)을 $s$에 배치하고 bin들에 discretizing하는 것이다.  

Neural Network의 제안된 architecture(특히, ReLU의 배치, 그림 2 참조)로, One-blob encoding은 네트워크의 선형 경로의 특정 부분을 효과적으로 차단하여 input의 다양한 sub-domain에서 Model을 전문화할 수 있게 한다. 연속 변수에 적용할 경우 quantization이 정보 손실을 초래하는 One-hot encoding과는 대조적으로, One-blob encoding은 손실없이 $s$의 정확한 위치를 찾아낸다.

### Analysis

## MONTE CARLO INTEGRATION WITH NICE


이 절에서는 Monte-Carlo estimation에 NICE framework를 적용한다. 목표는 integrand의 관찰에서 sampling PDF들을 추출하여 추정 분산을 줄이는 것이다. Sampling을 하기 위해 학습해야 할 PDF $q(x ; \theta)$와 integrand의 Ground-truth distribution $p(x)$에 대해 명시함으로써 eq(3)으로부터 아래와 같이 Monte-Carlo Estimator를 다시 작성할 수 있다.

$$\left<F\right>_N = \frac{1}{N} \sum_{i=1}^N \frac{f(X_i)}{q(X_i;\theta)} = \frac{1}{N} \sum_{i=1}^{N} \frac{p(X_i)\space F}{q(X_i; \theta)} \tag{18}$$

이상적인 케이스 $q(x; \theta) = p(x)$일 때, Estimator는 $F$의 정확한 값을 반환한다. 여기서 우리의 목표는 NICE를 활용하여 데이터로부터 $q$를 학습하는 동시에 Coupling layer들에서 Neural Network들을 $p$와 $q$ 사이의 거리가 최소화되도록 최적화하는것이다.

우리는 일반적으로 사용되는 divergence metric들 중 하나를 사용하여 거리를 정량화하는 표준적인 접근방식을 따른다. 모든 divergence 지표는 두 분포가 동일할 경우 최소치에 도달하지만, 모양은 다르며 따라서 실제로 다른 $q$를 생성한다.  
Section 1.3.1에서는 일반적인 Kullback-Leibler(KL) divergence을 이용하여 최적화한다. 여기서 더 나아가 Section 1.3.2에서 결과로 나오는 MC Estimator의 분산을 직접 최소화하는 것을 고려하고 그것이 $\chi^2$ divergence를 최소화하는 것과 같다는 것을 증명한다.

### Minimizing Kullback-Leibler Divergence

Deep Neural Network에 기반한 대부분의 생성 모델은 data points $x$의 likelihood $q(x; \theta)$을 정확하거나 효율적으로 평가할 수 없다. 그러나 여기서는 그러한 평가를 쉽게하는 다루기 쉬운 Jacobian determinant들과의 전단사 mapping에 기초한다.  
아래에서는 Gradient descent로 KL divergence를 최소화하는 것이 weighted log-likelihood를 최대화하는 것임을 보여준다.


\begin{align}
D_{KL}(p\space||\space q;\space \theta) & = \int_{\Omega}p(x) \space log \frac{p(x)}{q(x;\theta)} dx \\
& = \int_{\Omega} p(x)\space log \space p(x) dx \underbrace{- \int_{\Omega} p(x)\space log \space q(x;\theta) dx}_{\text{Cross entropy}} \tag{19}\\
\end{align}


Gradient Descent로 $D_{KL}$을 최소화하기 위해서는 학습 가능한 parameter들 $\theta$에 대한 gradient가 필요하다. 이것들은 Cross entropy term에서만 나타난다.  
그러므로,


\begin{align}
\nabla_{\theta} D_{KL}(p\space||q;\theta) & = - \nabla_\theta \int_{\Omega} p(x) \space log \space q(x;\theta)\space dx \tag{20}\\
& = E_q \left[ - \frac{p(X)}{q(X;\theta)} \nabla_{\theta} log \space q(X;\theta) \right] \tag{21}
\end{align}


여기서 기댓값은 $q$에서 sampling된 $X \sim q(x;\theta)$에 대한 기댓값이다. 즉, sample들은 학습된 생성 모델로부터 추출된다.  

대부분의 적분 문제에서 $p(x)$는 $f(x): p(x) = \frac{f(x)}{F}$ 를 통해 normalized되지 않은 형태로만 접근할 수 있다. 

$F$는 알 수 없기 때문에(애초에 추정하고자 하는), Gradient는 global scale factor $F$까지만 추정할 수 있다.  
Adam과 같은 일반적인 Gradient-descent 기반의 최적화 기법은 gradient 분산의 reciprocal 제곱근에 의해 step size를 스케일링하여 F를 cancelling하므로 이것은 문제가 아니다.  
**(공부 필요)**

더욱이 $f(x)$를 Monte-Carlo를 통해서만 추정할 수 있다면, 기대값의 선형성 때문에 Gradient는 여전히 정확하다. 따라서 eq(21)은 Gradient descent를 통해 $D_{KL}$를 최소화하는 것이, MC 추정치 $F$에 의해 weighted된 Negative log-likelihood를 최소화하는 것과 동일하다는 것을 보여준다.  
**(공부 필요)**

###  Minimizing Variance via $\chi^2$ Divergence

(unbiased) MC Integration의 맥락에서 가장 최소화해야하는 것은 Estimator의 분산이다.  

분산을 직접 최소화하기 위해 노력하는 이전 작품[Pantaleoni and Heitz 2017; Vévoda et al. 2018]에 영감을 받아 $X \sim q(x; \theta)$와 함께 $X \sim q(x; \theta)$를 사용하는 MC Estimator $\frac{p(X)}{q(X;\theta)}$에 대해 어떻게 이것이 달성될 수 있는지 Gradient descent를 통해 증명해본다.  

아래와 같이 분산을 정의하고 단순화해보자.

$$
\begin{align}
Var\left[ \frac{p(X)}{q(X;\theta)}\right] & = E \left[ \frac{p(X)^2}{q(X;\theta)^2} \right] - E\left[ \frac{p(x)}{q(X;\theta)} \right]^2 \\
& = \int_{\Omega}\frac{p(x)^2}{q(x;\theta)}\space dx - \underbrace{\left(\int_{\Omega} p(x) \space dx \right)^2}_{1} \tag{22}
\end{align}
$$

Gradient descent를 위한 분산의 stochastic gradient는 그러므로,

$$
\begin{align}
\nabla_{\theta} Var \left[\frac{p(x)}{q(X;\theta)} \right] & = \nabla_{\theta} \int_{\Omega}\frac{p(x)^2}{q(x;\theta)}\space dx \\\\
& = \int_{\Omega}p(x)^2 \nabla_\theta \frac{1}{q(x;\theta)} \space dx \\\\
& = \int_{\Omega}-\frac{p(x)^2}{q(x;\theta)}\space \nabla_{\theta}\space log \space q(x;\theta) \space dx \\\\
& = E \left[-\left(\frac{p(X)}{q(X;\theta)} \right)^2 \space \nabla_{\theta} \space log \space q(X;\theta)\right] \tag{23} \\
\end{align}
$$

자세히 살펴보면 eq(22)의 variance obsjective는 아래와 같이 Pearson $\chi^2$ divergence $D_{\chi^2}(p\space||\space q;\space \theta)$와 같다.

$$
\begin{align}
D_{\chi^2}(p\space||\space q;\space \theta) & = \int_{\Omega}\frac{(p(x)-q(x;\theta))^2}{q(x;\theta)}\space dx \\
&= \int_{\Omega} \frac{p(x)^2}{q(x;\theta)}\space dx - \underbrace{\left( 2 \int_{\Omega}p(x) \space dx - \int_{\Omega}q(x;\theta)\space dx\right)}_{1} \tag{24}
\end{align}
$$

이처럼 MC estimator의 분산을 최소화하는 것은 Ground truth와 학습된 분포 사이의 Pearson $\chi^2$ divergence를 최소화하는 것에 해당한다.

$KL$ divergence와 $\chi^2$ divergence의 gradient는 log-likelihood에 적용되는 weight에서만 다르다.  

$\nabla_\theta D_{KL}$에서 log-likelihood는 MC weight에 의해 weighted되는 반면, $\nabla_\theta D_{\chi^2}$를 최적화할 때 log-likelihood는 MC weight의 제곱에 의해 weighted된다.  

이는 두 loss function의 차이를 암시한다. $\chi^2$ divergence는 큰 불일치에 대해서(특히 $p$가 큰 density를 갖는 지역에서 낮은 $q$값을 지닐 때) 강한 페널티를 주게 된다. 이렇게 $\chi^2$ divergence는 $D_{KL}$보다 더 보수적인 q를 만들게 되는데, 우리가 Section 1.4에서 보듯 약간 더 안 좋은 평균 성능의 비용으로도 적은 outlier의 수를 관찰한다. (?)

## NEURAL PATH SAMPLING AND PATH GUIDING


이 절에서는 piecewise-polynomial warps(2절)과 함께 NICE(1절)를 취하고 light transfortation의 sequential MC 적분에 3절에 설명한 방법론을 적용한다. 우리의 목표는 즉각적으로 학습된 sampling density를 사용하여 light path들의 구축을 "Guide"함으로써 추정 분산을 줄이는 것이다.

이 절에서 다음과 같은 두 가지 다른 설정을 탐구해보고자 한다:  

1. light transportation의 Path-integral formulation을 활용하고 Primary Sample Space(PSS)에서 고차원 sampling을 사용하여 완전한 light path sample(4.1절)을 작성하는 **글로벌 설정**  

2. Integration이 2D(반)구형 도메인에 걸쳐 있고 path가 점진적으로 만들어지는 rendering equation에 자연스럽게 적용되는 **로컬 설정**(4.2).

### Primary-Sample-Space Path Sampling

복사 휘도 : 빛을 비추는 대상에서의 밝은 정도  
처리량(throughput) : 지정된 시간 내에 전송된 정보량

이미지를 생성하기 위해 renderer는 scene을 통해 가능한 path를 취한 후 카메라에 도달하는 빛의 양을 추정해야 한다. 이 전송은 Path integral formulation[Veach 1997]을 사용하여 공식화할 수 있으며, 여기서 센서(예: 픽셀)에 대한 복사휘도(radiance) 측정값 $I$은 path space $\mathcal P$에 대한 Path integral로 주어진다.

$$I = \int_{\mathcal P} L_e(x_0, x_1) \space T(\bar{x}) \space W(x_{k-1}, x_k) \space d\bar{x} \tag{25}$$

여기서,

위치 체인 $\bar{x} = x_0 \cdots x_k$는 $k$ 꼭지점을 가진 단일 광 경로를 나타낸다.  
경로 처리량 $T(\bar{x})$는 복사휘도를 전달하는 단일 광 경로 $\bar{x}$의 능력을 정량화한다.  
$L_e$는 방출된 복사휘도(emitted radiance)을 나타내며 $W$는 입사 복사 휘도(incident radiance)의 한 단위에 대한 센서 응답이다.

복사휘도 측정값 $I$는 MC estimator(3)의 형식으로 다음과 같이 추정될 수 있다.

$$\left<I\right> = \frac{1}{N} \sum_{j=1}^{N} \frac{L_e(x_{j0}, x_{j1}) \space T(\bar{x}_j) \space W(x_{j k-1}, x_{jk})}{q(\bar{x}_j)} \tag{26}$$

여기서,  

$q(\bar{x})$는 단일 광 경로 $\bar{x}$의 모든 $k$ 꼭지점들을 생산하는 joint probability density이다.

그러나 Joint probability density에서 직접 샘플을 그리는 것은 꼭지점의 제한된 특성(예: 표면에 있어야 함**(? 꼭지점을 생성하는 양 옆의 다른 꼭지점이 존재해야 한다는 것을 의미하는 것 같음)** ) 때문에 어렵다.

따라서 몇 가지 접근방식 [Guo et al. 2018; Kelemen et al. 2002]은 단위 하이퍼큐브 $\mathcal U$로 표현되는 Primaty Sample Space(PSS)에서 작동을 제안한다. 한 path는 표준 경로 구성 기법들 중 하나 $\rho$(즉. camera tracing)를 사용하여 random number들의 벡터 $z ∈ \mathcal U$를 변환함으로써 얻어진다.(예: 카메라 추적(camera tracing))  

$$\bar{x} = \rho(z)$$

### Path Guiding

### Experimental Setup

## Results